# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# import libraries
import pandas as pd
import numpy as np
import re
import sqlite3
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils.multiclass import type_of_target

In [22]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseData.db')
df = pd.read_sql('SELECT * FROM "disaster_response"', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.drop(columns=['id'], inplace=True)

In [24]:
X = df.message.values
y = df.select_dtypes(include=np.number).values

In [25]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'], dtype=object)

In [26]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [27]:
category_names = list(df.select_dtypes(include=np.number).columns)

In [28]:
def load_data():
    
    """
    Loading transformed data of Disaster Response and split into X and y.
  
    This function connects to the sqlite server where the ETL-Disaster Response Data is stored, loads the dataset, 
    removes the column "id" and splits the remaining columns into messages (X) and number values (y).
      
    Parameters:
    None
  
    Returns:
    array (X, y): messages and number values.
  
    """
    
    engine = create_engine('sqlite:///DisasterResponseData.db')
    df = pd.read_sql('SELECT * FROM "disaster_response"', engine)
    df.drop(columns=['id'], inplace=True)
    X = df.message.values
    y = df.select_dtypes(include=np.number).values
    return X, y

### 2. Write a tokenization function to process your text data

In [29]:
tokens = word_tokenize(X[0].lower().strip())
tokens = [w for w in tokens if w not in stopwords.words("english")]
lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]#
lemmed

['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [30]:
def tokenize(text):
    
    """
    Transforming string text into arrays of words.
  
    This function splits strings into words, removes stopwords and lemmatize the words.
      
    Parameters:
    text (string): string input containing text.
  
    Returns:
    lemmed (array): array with separate text strings for machine learning model input.
  
    """
    
    tokens = word_tokenize(text.lower().strip())
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
def model_pipeline():
    #
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    return pipeline
pipeline = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [39]:
X,y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
type_of_target(y_test)
type_of_target(y_pred)
target_names = np.unique(y_test)
report = classification_report(np.hstack(y_test),np.hstack(y_pred))
    
print('Target names:', target_names)
print('Classification report:\n', report)

Target names: [0 1 2]
Classification report:
              precision    recall  f1-score   support

          0       0.94      0.99      0.97    215615
          1       0.78      0.33      0.46     20280
          2       0.73      0.33      0.45        49

avg / total       0.93      0.93      0.92    235944



In [15]:
def display_results(y_test, y_pred):
    
    """
    Displaying model results.
  
    This function displays the f1 score, precision and recall in the classification report of the predicted output.
    It shows how well the selected prediction model works.
      
    Parameters:
    y_test: actual results from the dataset, created within train_test_split.
    y_pred: predicted results in Model.
  
    Returns:
    classification report: shows precision, recall, f1-score and support on different target names.
      
    """
    
    type_of_target(y_test)
    type_of_target(y_pred)
    target_names = np.unique(y_test)
    report = classification_report(np.hstack(y_test),np.hstack(y_pred))
    
    print('Target names:', target_names)
    print('Classification report:\n', report)

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
list(pipeline.get_params())

['memory',
 'steps',
 'vect',
 'tfidf',
 'clf',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__strip_accents',
 'vect__token_pattern',
 'vect__tokenizer',
 'vect__vocabulary',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'clf__estimator__algorithm',
 'clf__estimator__leaf_size',
 'clf__estimator__metric',
 'clf__estimator__metric_params',
 'clf__estimator__n_jobs',
 'clf__estimator__n_neighbors',
 'clf__estimator__p',
 'clf__estimator__weights',
 'clf__estimator',
 'clf__n_jobs']

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fdbf071dc80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
              metric_params=None, n_jobs=1, n_neighbors=5, p=2,
              weights='uniform'),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_fea

In [21]:
parameters = {
    'clf__estimator__leaf_size': [20, 30, 40],
    'clf__estimator__metric_params': (None, 50, 100),
    'clf__estimator__n_jobs': [1, 2, 3],
    'clf__estimator__n_neighbors': [4, 5, 6],
    'clf__estimator__p':[1, 2, 3],
}

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
def model_pipeline_2():
    #
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    parameters = {
        'clf__estimator__leaf_size': [20, 30, 40],
        'clf__estimator__metric_params': (None, 50, 100),
        'clf__estimator__n_jobs': [1, 2, 3],
        'clf__estimator__n_neighbors': [4, 5, 6],
        'clf__estimator__p':[1, 2, 3],
    }
    cv = GridSearchCV(pipeline, param_grid = parameters)
    return cv

In [23]:
def main_2():
    #
    X,y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = model_pipeline_2()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    display_results(y_test, y_pred)

In [ ]:
main_2()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
def main():
    #
    X,y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    display_results(y_test, y_pred)

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.